In [19]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpmath import nsum, exp, inf
from main import get_all_retention_rates_for_cohort, get_data

# 4a

In [61]:
data = get_data()

retention_rate = 0.56530017

#E[lifetime] = 1*retention_rate + 2*retention_rate^2 +..n*retention_rate^3

print(nsum(lambda n: n*retention_rate**n, [1, inf]))

2.9915769295055


# 4b

>>Should develop strategies to keep the customers in the second month

The expected lifetime of above 2 is heavily influenced by the loyal customers, that have been staying for many months and therefore driving the expected value up. On the other side we have seen big drops of customers after only one or two months, and we would therefore want to get more users past that critical point of two months. This could be done by more effectively showing the customer relevant content. It is however difficult to personalize content for a user that they have little data on, and one strategy could therefore be to gather data about the particular user from outside of Netflix itself to propose relevant, personalized content that may make the user stay past the critical point

# 4c

In [62]:

def get_all_retention_rates_time_normalized(data):
    retention_rates_all_cohorts = []
    for i in range(12):
        all_retention_rates_for_cohort = (get_all_retention_rates_for_cohort(data,i))
        retention_rates_all_cohorts.append(all_retention_rates_for_cohort)
    for j in range(1,len(retention_rates_all_cohorts)):
        for i in range(j,len(retention_rates_all_cohorts[j])):
            retention_rates_all_cohorts[j][i-j] = retention_rates_all_cohorts[j][i]
            retention_rates_all_cohorts[j][i] = np.NaN
    return retention_rates_all_cohorts

get_all_retention_rates_months_after_acquisition = get_all_retention_rates_time_normalized(data)

avg_monthly_retention_rate_across_cohorts = np.apply_along_axis(np.nanmean,axis = 0, arr = get_all_retention_rates_months_after_acquisition)


retention_probabilities = np.cumprod(avg_monthly_retention_rate_across_cohorts[1:-1])
print(retention_probabilities)

expected_value = 0
i=1
for prob in list(retention_probabilities):
    expected_value += prob*i
    i+=1
print(expected_value)


[0.48308715 0.05753459 0.01415235 0.00602463 0.0026858  0.00161148
 0.00053716 0.00053716 0.00053716 0.00053716]
0.7060732139754536


/opt/homebrew/lib/python3.10/site-packages/numpy/lib/shape_base.py:379: RuntimeWarning: Mean of empty slice
  res = asanyarray(func1d(inarr_view[ind0], *args, **kwargs))
/opt/homebrew/lib/python3.10/site-packages/numpy/lib/shape_base.py:402: RuntimeWarning: Mean of empty slice
  buff[ind] = asanyarray(func1d(inarr_view[ind], *args, **kwargs))


In [63]:
expectation_value = np.arange(1,len(retention_probabilities)+1,1)@retention_probabilities
expectation_value

0.7060732139754536

# 4d

The values for the different methods of calculating are radically different. Similarly as before we see here a sort of "abuse" of doing the average of averages in eliminating the relevance of the sample size. In this case in 4a we are using the average retention rate of the average retention rates of each month after customer acquisition.
To get the relevance of all customers, also the ones that churned early, the last retention rate in 4c should be used as it is more realistic.
Using this, we realize that many churn after only one month, and the active probabilities fall quickly

In [41]:
#4d
#Strategies for keeping customers during first month vs second month